In [1]:
import pandas as pd
import numpy as np
import pprint
from sklearn.cluster import DBSCAN
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200 
pd.options.display.max_colwidth = 100

# Exploration

In [2]:
data = pd.read_csv('../data/processed/data.csv')

In [3]:
def get_context(data, symbol, columns = data.columns.tolist()):
    idx_mask = data.symbol == symbol
    return data.loc[idx_mask, columns]

In [4]:
context=[
    'symbol', 
    'calendarYear', 
    'target', 
    'revenue',
    'totalAssets', 
    'netIncome', 
    'totalEquity',
    'totalLiabilitiesAndTotalEquity',
    'finalLink'
    ]

In [9]:
data[(data.shortTermDebt==0)&(data.longTermDebt == 0)]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target
5,A,2005,3.02004,0.03392,3.05396,1.02160,0.97955,0.97819,6.03330,1.18441,0.00000,0.49113,0.49113,0.00000,0.00000,1.45033,3.12587,0.00000,9.15917,0.46671,0.00000,0.00000,0.33511,1.82478,2.62660,0.00000,0.00000,0.00000,0.99583,0.99583,0.00000,0.00000,3.62242,0.00000,0.00678,-1.98487,-0.06648,7.58131,5.53674,9.15917,0.00000,5.53674,9.15917,0.00000,0.00000,-3.02004,6.97215,3.55052,3.42163,0.00000,1.00125,2.17481,0.00000,2.17481,0.00000,3.17606,6.72658,0.00000,0.00000,0.25235,0.90628,0.00000,0.24556,0.00000,0.09904,0.34460,0.00000,0.21029,0.44365,0.00000,0.00000,0.00000,0.67022,0.67836,2005-10-31,USD,1090872.00000,2006-01-17,2006-01-17 16:47:54,FY,-0.01628,0.00000,-0.26456,0.00000,0.00000,3.40671,0.80453,1.21968,-0.18858,-0.08683,-0.05156,0.04070,0.01492,-0.27134,0.00000,0.26863,-0.39345,0.00000,-0.93342,-1.05824,-0.01085,-0.12075,3.02004,3.14079,1.21968,-0.18858,1.03110,https://www.sec.gov/Archives/edgar/data/1090872/000104746906000516/a2166540z10-k.htm,https://www.sec.gov/Archives/edgar/data/1090872/000104746906000516/a2166540z10-k.htm,10.00000,1.35671,2006-01-27,2006.00000,4.00000,16.05531
44,AAPL,2004,4.12824,3.46917,7.59742,1.45997,0.14044,0.61180,9.80962,0.98305,0.11124,0.02364,0.13487,0.00000,0.11958,0.14600,1.38350,0.00000,11.19312,2.01754,0.00000,0.00000,0.75640,0.95246,3.72640,0.00000,0.00000,0.40879,0.00000,0.40879,0.00000,0.00000,4.13520,0.00000,3.49559,3.71250,-0.02086,-0.12931,7.05792,11.19312,0.00000,7.05792,11.19312,0.00000,0.00000,-4.12824,11.51153,8.37051,3.14103,0.00000,0.67993,0.00000,0.00000,1.97583,0.00000,2.65576,11.02626,0.08899,0.00417,0.20857,0.74528,0.00000,0.45329,0.00000,0.07926,0.53254,0.00000,0.14878,0.38376,0.00000,0.00000,0.00000,28.93392,30.15804,2004-09-25,USD,320193.00000,2004-12-03,2004-12-02 18:05:49,FY,0.02781,0.04588,0.62292,-0.01112,0.41296,-0.05423,0.00973,1.29868,-0.24472,0.00000,

In [31]:
data[data.totalAssets == 0]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target


In [32]:
data[data.revenue == 0]

,symbol,calendarYear,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,goodwillAndIntangibleAssets,longTermInvestments,taxAssets,otherNonCurrentAssets,totalNonCurrentAssets,otherAssets,totalAssets,accountPayables,shortTermDebt,taxPayables,deferredRevenue,otherCurrentLiabilities,totalCurrentLiabilities,longTermDebt,deferredRevenueNonCurrent,deferredTaxLiabilitiesNonCurrent,otherNonCurrentLiabilities,totalNonCurrentLiabilities,otherLiabilities,capitalLeaseObligations,totalLiabilities,preferredStock,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,minorityInterest,totalEquity,totalLiabilitiesAndTotalEquity,totalInvestments,totalDebt,netDebt,revenue,costOfRevenue,grossProfit,grossProfitRatio,researchAndDevelopmentExpenses,generalAndAdministrativeExpenses,sellingAndMarketingExpenses,sellingGeneralAndAdministrativeExpenses,otherExpenses,operatingExpenses,costAndExpenses,interestIncome,interestExpense,depreciationAndAmortization,ebitda,ebitdaratio,operatingIncome,operatingIncomeRatio,totalOtherIncomeExpensesNet,incomeBeforeTax,incomeBeforeTaxRatio,incomeTaxExpense,netIncome,netIncomeRatio,eps,epsdiluted,weightedAverageShsOut,weightedAverageShsOutDil,date,reportedCurrency,cik,fillingDate,acceptedDate,period,deferredIncomeTax,stockBasedCompensation,changeInWorkingCapital,accountsReceivables,accountsPayables,otherWorkingCapital,otherNonCashItems,netCashProvidedByOperatingActivities,investmentsInPropertyPlantAndEquipment,acquisitionsNet,purchasesOfInvestments,salesMaturitiesOfInvestments,otherInvestingActivites,netCashUsedForInvestingActivites,debtRepayment,commonStockIssued,commonStockRepurchased,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink,month,inflationMultiplier,mcapDate,mcapYear,mcapWeek,target


###  DBSCAN for outliers

In [ ]:
dbscan = DBSCAN(eps=75, min_samples=5)
dbscan.fit(data)

DBSCAN(eps=75)

# Maybe check increases

In [ ]:
also_compute = [
    'weightedAverageShsOut',
    'weightedAverageShsOutDil',
    'preferredStock',
    'commonStock',
]
for col in currency_features + also_compute:
    yoy_colname = col + '_yoy'
    data[yoy_colname] = data[col].pct_change(1)